# Проект большие языковые модели

### Подготовка окружения

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFD, Lowercase, StripAccents, Sequence
from tokenizers.processors import TemplateProcessing
from transformers import PreTrainedTokenizerFast, LlamaConfig, LlamaForCausalLM, Trainer, TrainingArguments,TrainerCallback, AutoModelForCausalLM, AutoTokenizer
from torch.optim import AdamW 
import os
import gc
import torch
import warnings
from torch.utils.data import DataLoader
import regex as re
import numpy as np
import random
from pathlib import Path
from datasets import Dataset
import unicodedata
from collections import Counter
from tqdm.notebook import tqdm

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True

seed_everything(42)
warnings.filterwarnings('ignore')

In [ ]:
DATA_DIR = Path("RussianNovels/corpus")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
VOCAB_SIZE = 3000
CYRILLIC_RE = re.compile(r"^[\p{IsCyrillic}\s0-9.,!?—–:;\"'()«»…\-]+$")
BATCH_SIZE = 2 if DEVICE=='cpu' else 8
MAX_EPOCHS = 1
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 0.01
GRAD_CLIP = 1.0
SAVE_PATH = "best_model.pt"

TEST_PROMPTS = [
    "Все мысли, которые имеют огромные последствия",
    "Сила войска зависит от его духа",
    "Мысль о том, что он принес страдания",
    "Человек сознает себя свободным",
    "Что бы ни случилось, я всегда буду",
    "Любовь мешает смерти",
    "Нет, жизнь не кончена",
    "Всякая мысль, даже самая простая",
    "Война не любезность, а самое гадкое дело",
    "Чтобы жить честно"
] 
MODEL_NAME = "Qwen/Qwen2.5-0.5B"

QUESTIONS = [
    "сколько планет в нашей солнечной системе?",
    "расскажи стих",
    "когда собирать крыжовник?",
    "Как быстро выучить новый язык?"
]

print(f"{DEVICE=}")

DEVICE='cpu'


In [4]:
!git clone https://github.com/JoannaBy/RussianNovels.git
!git clone https://huggingface.co/datasets/d0rj/alpaca-cleaned-ru

fatal: destination path 'RussianNovels' already exists and is not an empty directory.
fatal: destination path 'alpaca-cleaned-ru' already exists and is not an empty directory.


## Pretrain

### Подготовка корпуска текстов

In [5]:
def collect_texts(root: Path):
    texts = []
    for path in root.rglob("*"):
        if path.suffix in {".txt"}:
            try:
                texts.append(path.read_text(encoding="utf-8"))
            except Exception:
                pass
    return texts

raw_texts = collect_texts(DATA_DIR)
raw_texts = list(set(raw_texts))
print(f"Файлов в корпусе: {len(raw_texts)}")

Файлов в корпусе: 107


In [6]:
def filter_cyrillic_sentences(text: str):
    '''фильтруем предложения с некириллическими символами'''
    sentences = re.split(r"[.!?]+", text)
    return [
        s.strip()
        for s in sentences
        if s.strip() and CYRILLIC_RE.match(s.strip())
    ]

def normalize_punctuation(text: str) -> str:
    '''нормализуемы пунктуацию'''
    text = re.sub(r"[!?]{2,}", r"\1", text)
    text = re.sub(r"\.{3,}", "…", text)
    text = re.sub(r",,{2,}", ",", text)
    text = re.sub(r"-{2,}", "-", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r'[«»]', '"', text)
    return text.strip()    

clean_sentences = []

for text in raw_texts:
    sentences = filter_cyrillic_sentences(text)
    sentences = [normalize_punctuation(s) for s in sentences]
    clean_sentences.extend(sentences)

print(f"Предложений после очистки: {len(clean_sentences)}")

Предложений после очистки: 523962


In [7]:
print("Проверка состава оставшихся спец символов")
def is_letter_or_digit(ch: str) -> bool:
    cat = unicodedata.category(ch)
    return cat.startswith("L") or cat.startswith("N")

counter = Counter()

for sent in clean_sentences:
    for ch in sent:
        if not is_letter_or_digit(ch):
            counter[ch] += 1


non_alnum = counter.most_common()
print(f"symb | freg    | name")
for ch, freq in non_alnum[:30]:
    name = unicodedata.name(ch, "UNKNOWN")
    print(f"'{ch}'  | {freq:>7} | {name}")

Проверка состава оставшихся спец символов
symb | freg    | name
' '  | 5716309 | SPACE
','  |  853248 | COMMA
'-'  |  279089 | HYPHEN-MINUS
'"'  |   49998 | QUOTATION MARK
':'  |   44922 | COLON
';'  |   32563 | SEMICOLON
'–'  |   24641 | EN DASH
')'  |    7897 | RIGHT PARENTHESIS
'('  |    7337 | LEFT PARENTHESIS
'—'  |    4435 | EM DASH
'…'  |    2988 | HORIZONTAL ELLIPSIS
'''  |    1152 | APOSTROPHE


In [8]:
# разбиение на чанки
chunks = []
current = ""

for sent in clean_sentences:
    max_chars = 2000  # ~512 токенов
    if len(current) + len(sent) < max_chars:
        current += " " + sent
    else:
        chunks.append(current.strip())
        current = sent

if current:
    chunks.append(current.strip())

print(f"Чанков: {len(chunks)}")
print(f"чанк  0: {chunks[0]}")
print(f"чанк 10: {chunks[10]}")


Чанков: 20036
чанк  0: Несмотря на то, что время клонилось къ вечеру, жаръ не спадалъ Такъ было тихо кругомъ ихъ, какъ-будто все замерло въ лѣсу Ни травка, ни одинъ листикъ не шелохнулись Лѣсъ такъ былъ густъ, что охотниковъ окружала со всѣхъ сторонъ плотная, высокая стѣна зелени, отнимавшая всякую возможность прохлады Одинъ изъ охотниковъ, щеголевато одѣтый, покрой платья котораго скорѣе былъ красивъ, чѣмъ удобенъ, снялъ бархатную небольшую фуражку и, тряхнувъ своими длинными, взмокшими волосами, съ сердцемъ сказалъ: - А все по твоей милости, я умираю отъ жажды и задыхаюсь отъ усталости Я говорилъ, взять провожатаго Нѣтъ, не согласился Вотъ теперь и блуждай по лѣсу И, взглянувъ на часы, онъ еще раздражительнѣе продолжалъ: - Скоро солнце сядетъ, а ужь ночью трудно по этому глухому лѣсу пробираться; я и такъ себѣ ноги всѣ разбилъ и все лицо перецарапалъ - Останемся ночевать, равнодушно замѣтилъ его товарищъ въ картузѣ съ козырькомъ, походившимъ на бекасиный носъ - Ночевать что ты, съ ум

### Обучение токенизатора

In [9]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

# Нормализация текста
tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])
# разбиение по пробелам 
tokenizer.pre_tokenizer = Whitespace() 
trainer = BpeTrainer(
    vocab_size=VOCAB_SIZE,
    min_frequency=2,
    special_tokens=["[PAD]", "[UNK]", "[BOS]", "[EOS]"] 
    )
tokenizer.train_from_iterator(chunks, trainer)
# (BOS/EOS) 
tokenizer.post_processor = TemplateProcessing(
    single="[BOS] $A [EOS]",
    pair="[BOS] $A [EOS] $B:1 [EOS]:1",
    special_tokens=[
        ("[BOS]", tokenizer.token_to_id("[BOS]")),
        ("[EOS]", tokenizer.token_to_id("[EOS]")), 
        ], 
        )
# Сохранение
tokenizer.save("bpe_ru_3k.json")

In [10]:
encoded = tokenizer.encode("Пример текста для проверки токенизатора")
print(encoded.tokens)
print(encoded.ids)


['[BOS]', 'пример', 'те', 'к', 'ста', 'для', 'про', 'вер', 'ки', 'то', 'ке', 'ни', 'за', 'тора', '[EOS]']
[2, 1903, 80, 31, 123, 308, 106, 170, 100, 58, 220, 70, 81, 1680, 3]


### Сборка датасета

In [11]:

hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="bpe_ru_3k.json",
    bos_token="[BOS]",
    eos_token="[EOS]",
    unk_token="[UNK]",
    pad_token="[PAD]",
)

def tokenize(batch):
    return hf_tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

def add_labels(batch):
    labels = []
    for ids, mask in zip(batch["input_ids"], batch["attention_mask"]):
        l = ids.copy()
        l = [
            token if m == 1 else -100
            for token, m in zip(l, mask)
        ]
        labels.append(l)

    batch["labels"] = labels
    return batch


dataset = Dataset.from_dict({"text": chunks})

dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns=["text"],
).map(add_labels, batched=True)

dataset.set_format(type="torch")

Map:   0%|          | 0/20036 [00:00<?, ? examples/s]

Map:   0%|          | 0/20036 [00:00<?, ? examples/s]

### Подготовка модели LlamaForCausalLM

In [12]:
config = LlamaConfig(
    hidden_size=1024,
    intermediate_size=1536,
    num_hidden_layers=16,
    num_attention_heads=16,
    num_key_value_heads=8,
    max_position_embeddings=2048,
    rms_norm_eps=1e-6,
    hidden_act="silu",
    tie_word_embeddings=True,
    attention_bias=False,
    rope_theta=10000.0,
    use_cache=False,     
)
config.vocab_size = max(hf_tokenizer.get_vocab().values()) + 1
config.pad_token_id = hf_tokenizer.pad_token_id
config.bos_token_id = hf_tokenizer.bos_token_id
config.eos_token_id = hf_tokenizer.eos_token_id

model = LlamaForCausalLM(config).to(DEVICE)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Обучаемых параметров: {num_params:,}")

Обучаемых параметров: 128,934,912


In [13]:
print("Проверим синхронизацию модели и текенизатора")
assert model.config.vocab_size == max(hf_tokenizer.get_vocab().values()) + 1
assert model.config.pad_token_id == hf_tokenizer.pad_token_id
assert model.config.eos_token_id == hf_tokenizer.eos_token_id
print("ок")

Проверим синхронизацию модели и текенизатора
ок


In [14]:
print("Sanity-check начальной модели на инференсе")
model.eval()

prompt = "Привет"
inputs = hf_tokenizer(
    prompt,
    return_tensors="pt",
    return_token_type_ids=False
).to(model.device)

with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=False,
        eos_token_id=hf_tokenizer.eos_token_id,
    )

print(hf_tokenizer.decode(out[0], skip_special_tokens=True))


Sanity-check начальной модели на инференсе
при вет кры кры брат брат брат брат брат брат брат брат бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары


In [15]:
# оптимизатор
def get_optim():
    return AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

optimizer = get_optim()

# освобождение памяти
def gc_collect():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize() 

In [16]:
print("Сheck сходимости модели при обучении на малом датасете (2 предожения)")
tiny_dataset = dataset.select(range(2))
tiny_loader = DataLoader(tiny_dataset, batch_size=1, shuffle=True)

model.train()

progress_bar = tqdm(
        range(201),
        desc=f"train check",
    )
best_loss = float("inf")

for step in progress_bar:
    batch = next(iter(tiny_loader))
    batch = {k: v.to(model.device) for k, v in batch.items()}

    loss = model(**batch).loss
    progress_bar.set_description(f"train check | loss: {loss:.2f}")
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if step> 0 and step % 20 == 0:
        progress_bar.write(f"epoch: {step:>3} loss: {loss.item():.2f}")
        if loss.item() < best_loss:
            best_loss = loss.item()
            torch.save(model.state_dict(), SAVE_PATH)
            progress_bar.write(f"best model saved with {best_loss=:.2f}")

del tiny_dataset, tiny_loader, optimizer
gc_collect()

Сheck сходимости модели при обучение на малом датасете (2 предожения)


train check:   0%|          | 0/201 [00:00<?, ?it/s]

epoch:  20 loss: 5.32
best model saved with best_loss=5.32
epoch:  40 loss: 5.04
best model saved with best_loss=5.04
epoch:  60 loss: 4.73
best model saved with best_loss=4.73
epoch:  80 loss: 4.40
best model saved with best_loss=4.40
epoch: 100 loss: 2.32
best model saved with best_loss=2.32
epoch: 120 loss: 0.12
best model saved with best_loss=0.12
epoch: 140 loss: 0.01
best model saved with best_loss=0.01
epoch: 160 loss: 0.00
best model saved with best_loss=0.00
epoch: 180 loss: 0.00
epoch: 200 loss: 0.01


In [19]:
print("Сheck инференса модели после обучения на малом датасете")
def check_inference(model, prompts: list):
    model.eval()
    for pr in prompts:
        inputs = hf_tokenizer(pr, return_tensors="pt").to(model.device)
        inputs.pop("token_type_ids", None)

        out = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True, 
        eos_token_id=hf_tokenizer.eos_token_id,
    )
        print("input:  ",  pr)
        print("output: ", hf_tokenizer.decode(out[0], skip_special_tokens=True))
        print()


check_inference(model, TEST_PROMPTS[:2])

Сheck инференса модели после обучения на малом датасете
input:   Все мысли, которые имеют огромные последствия
output:  все мысли , которые име ют огром ные послед ствия къ вече ру , жар ъ не спа дал ъ не х ъ их ъ их ъ их ъ , какъ - будто все заме р ло въ л ѣ су ни тра в ка , ни один ъ ли сти къ не ше ло х нулись л ѣ съ

input:   Сила войска зависит от его духа
output:  сила вои ска зави си т от его ду ха время кло кло , жар ъ не спа дал ъ не въ л ѣ су ни с ха ст ѣ ле ъ его заме р ло въ л ѣ су ни тра в ка , ни один ъ ли сти къ не ше ло х нулись л ѣ съ так



### Обучение модели LlamaForCausalLM

In [34]:
# Callback для валидации процесса обучения
class PromptCallback(TrainerCallback):
    def __init__(self, tokenizer, prompts, max_new_tokens=50):
        self.tokenizer = tokenizer
        self.prompts = prompts
        self.max_new_tokens = max_new_tokens
    def on_evaluate(self, args, state, control, **kwargs):
        model = kwargs.get("model")
        if model is None:
            return
        model.eval()

        for prompt in self.prompts:
            
            inputs = self.tokenizer(prompt, return_tensors="pt").to(model.device)
            
            inputs.pop("token_type_ids", None)
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=self.max_new_tokens
                )

            decoded = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            print("input:  ",  prompt)
            print("output: ", decoded)
            print()


In [35]:
# обучение модели

split_dataset = dataset.select(range(2000)).train_test_split(test_size=0.01)
train_dataset = split_dataset['train']
val_dataset = split_dataset['test']

training_args = TrainingArguments(
    output_dir="./checkpoints",
    num_train_epochs=MAX_EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=16,  # 16 × 8 = 128
    weight_decay=WEIGHT_DECAY,
    eval_strategy="steps",
    save_strategy="no",
    eval_steps=1 if DEVICE=='cpu' else 100,
    logging_steps=1 if DEVICE=='cpu' else 100,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=hf_tokenizer,
    callbacks=[
        PromptCallback(
            tokenizer=hf_tokenizer,
            prompts=TEST_PROMPTS[:2]
        )
    ]
)
trainer.train()

Step,Training Loss,Validation Loss
1,6.706500,6.915845


input:   Все мысли, которые имеют огромные последствия
output:  все мысли , которые име ют огром ные послед ствия , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что

input:   Сила войска зависит от его духа
output:  сила вои ска зави си т от его ду ха , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что



KeyboardInterrupt: 

In [ ]:
# train_loader = DataLoader(
#     dataset,
#     batch_size=BATCH_SIZE,
#     shuffle=True
# )
# model.train()

# optimizer = get_optim()

# total_steps = len(train_loader) * MAX_EPOCHS

# scheduler = get_linear_schedule_with_warmup(
#     optimizer,
#     num_warmup_steps=total_steps // 10, 
#     num_training_steps=total_steps
# )

# best_loss = float("inf")

# for epoch in range(MAX_EPOCHS):

#     progress_bar = tqdm(train_loader)

#     total_loss = 0.0
#     for step, batch in enumerate(progress_bar):
#         batch = {k: v.to(DEVICE) for k, v in batch.items()}

#         outputs = model(
#             input_ids=batch["input_ids"],
#             attention_mask=batch["attention_mask"],
#             labels=batch["labels"]
#         )
#         loss = outputs.loss

#         progress_bar.set_description(
#             f"train epoch {str(epoch+1):<2} step {str(step+1):<3} loss: {loss.item():.2f}"
#         )
#         loss.backward()
        
#         torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)

#         # Optimizer step
#         optimizer.step()
#         scheduler.step()
#         optimizer.zero_grad()

#         total_loss += loss.item()

#         if step>0 and step % 100 == 0:
#             progress_bar.write(f"epoch {epoch+1} step {step+1} loss: {loss.item():.2f}")

#     avg_loss = total_loss / len(train_loader)

#     progress_bar.write(f"epoch {epoch+1} finished. avg_loss: {avg_loss:.2f}")
    
#     if avg_loss < best_loss:
#         best_loss = avg_loss
#         torch.save(model.state_dict(), SAVE_PATH)
#         progress_bar.write(f"best model saved with {best_loss=:.2f}")

In [25]:
gc_collect()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)

model_qwen = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)

In [ ]:
def generate_answer(question):
    prompt = (
        "### Инструкция:\n"
        f"{question}\n\n"
        "### Ответ:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model_qwen.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)



for q in QUESTIONS:
    continue
    print("=" * 60)
    print("Вопрос:", q)
    print("Ответ:")
    print(generate_answer(q))


In [26]:
len(dataset)

20036